# A Practical Guide to Coding with Databases

Betsy Alpert

Console (console.to)



@betsybookworm

# What even is this talk
I need to code things and use a database!
* Where do I start
* What do I use
* What do I need to think about

# Which database?
There are SO MANY

## Different databases for different purposes
* Relational all-the-things
* nosql ???
* Time-series databases
* Graph databases
* many more...

# What's your favourite database?

# PostgreSQL aka postgres aka it's just nice
Here's one I prepared earlier...

# Which language are we going to use?

# Python aka Python 3 aka it's just nice
But the concepts are the same across languages

# What now?
Actually talk to the database what?

# Setting things up
You need:
* Your database
* Your environments
  * Development
  * Where your system will live or how it will be distributed
* A (good) way to get from one to the other
  * Actual physical/network connection
  * Database connection details

In [ ]:
% set_env DB_HOST 172.16.255.2
% set_env DB_USER postgres
% set_env DB_PASSWORD postgres
% set_env DB_PORT 5432
% set_env DB_NAME postgres

from os import getenv

# Database libraries
* Database API (https://www.python.org/dev/peps/pep-0249/)
* More abstracted libraries
* ORM

# Code time!

# Psycopg2
## Postgres DB API for Python

In [ ]:
from psycopg2 import connect

connection = connect(database=getenv('DB_NAME'), 
                     user=getenv('DB_USER'), 
                     password=getenv('DB_PASSWORD'), 
                     host=getenv('DB_HOST'), 
                     port=getenv('DB_PORT'))
print(connection)

cursor = connection.cursor()

In [ ]:
cursor.execute('''
    CREATE TABLE trainer
    (id INT PRIMARY KEY NOT NULL,
    name TEXT,
    username TEXT NOT NULL,
    team TEXT);
''')

cursor.execute('''
    INSERT INTO trainer (id, name, username, team) 
    VALUES (1, 'Betsy', 'betsybookworm', 'yellow');
''')

connection.commit()

In [ ]:
# add books with loop

In [ ]:
cursor.execute("SELECT * FROM trainer;")

print('{} rows found'.format(cursor.rowcount))

rows = cursor.fetchall()
for row in rows:
    print(row)

In [ ]:
connection.close()
print(connection)

## SQLAlchemy

* ORM, session management etc
* Can be used with a variety of databases, not just PostgreSQL
* Uses database drivers such as psycopg2 in the background


In [ ]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, Text, ForeignKey
from sqlalchemy.orm import relationship

In [ ]:
MonsterBase = declarative_base()

class MonsterType(MonsterBase):
    __tablename__ = 'monster_type'
    
    id = Column(Integer, primary_key=True)
    name = Column(Text, nullable=False)
    monsters = relationship('PocketMonster', back_populates='type_')

class PocketMonster(MonsterBase):
    __tablename__ = 'pocket_monster'
    
    id = Column(Integer, primary_key=True)
    name = Column(Text, nullable=False)
    type_id = Column(Integer, ForeignKey('monster_type.id'), nullable=False)
    type_ = relationship('MonsterType', back_populates='monsters')
    
    def __init__(self, name=None, type_=None):
        """
        Add a new type of Pocket Monster
        
        :param name: What this monster is called
        :type name: string
        :param type_: What type of monster is it?
        :type type_: MonsterType | int
        """
        self.name = name   
        self.type_id = type_.id if isinstance(type_, MonsterType) else type_
    
    def __repr__(self):
        return 'PocketMonster<{name!s} ({type_!s})>'\
                .format(name=self.name, type_=self.type_.name)

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}'.format(
            username=getenv('DB_USER'),
            password=getenv('DB_PASSWORD'),
            host=getenv('DB_HOST'),
            port=getenv('DB_PORT'),
            database=getenv('DB_NAME')
        ))

MonsterBase.metadata.create_all(engine)

session_maker = sessionmaker(bind=engine)
db_session = session_maker()

print('Pocket Monsters!')
print(db_session.query(PocketMonster).all())
print(db_session.get_bind())

In [ ]:
# Monster types
grass_type = MonsterType(name='Grass')
lightning_type = MonsterType(name='Lightning')
water_type = MonsterType(name='Water')

db_session.add_all([grass_type, lightning_type, water_type])

db_session.commit()

# Monsters!
flowerasaur = PocketMonster(name='Flowerasaur', type_=grass_type)
chooseachu = PocketMonster(name='Chooseachu', type_=lightning_type)
splashle = PocketMonster(name='Splashle', type_=water_type)
magiflop = PocketMonster(name='Magiflop', type_=water_type)

db_session.add_all([flowerasaur, chooseachu, splashle, magiflop])

db_session.commit()

In [ ]:
print(chooseachu)

print(chooseachu.type_.name)

In [ ]:
query = db_session.query(PocketMonster)\
                  .filter(PocketMonster.type_id == MonsterType.id, 
                          MonsterType.name == 'Grass')
    
print(query.statement)
print('---')
print(query.all())

In [ ]:
db_session.close()

# You can write nice things too

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

class SessionManager(object):
    """
    Creates engines and sessions for SQLAlchemy ORM
    """
    def __init__(self, host=None, port=None, user=None, password=None, 
                 database=None):
        self.engine = create_engine(
            'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}'.format(
                username=user,
                password=password,
                host=host,
                port=port,
                database=database)
        )
        
        self._sessionmaker = sessionmaker(bind=self.engine
                                         )  
    def __enter__(self):
        """
        :return: Database session
        :rtype: sqlalchemy.orm.session.Session
        """
        self._session = self._sessionmaker()
        return self._session
    
    def __exit__(self, *args):
        # Could catch exceptions and rollback etc
        self._session.commit()
        self._session.close()

In [ ]:
monster_sessions = SessionManager(
    host=getenv('DB_HOST'),
    user=getenv('DB_USER'),
    password=getenv('DB_PASSWORD'),
    port=getenv('DB_PORT'),
    database=getenv('DB_NAME')
)

In [ ]:
with monster_sessions as session:
    monsters = session.query(PocketMonster).all()
    print(monsters)

In [ ]:
with monster_sessions as session:
    all_types = session.query(MonsterType).all()
    
    for monster_type in all_types:
        print('{} Monsters:'.format(monster_type.name))
        
        for monster in monster_type.monsters:
            print('* {}'.format(monster.name))

These examples based on code at: github.com/betsybookwyrm/python-cafe-sqlalchemy

# Let's break things

In [ ]:
from psycopg2 import connect, ProgrammingError

# Back to psycopg2
connection = connect(database=getenv('DB_NAME'), 
                     user=getenv('DB_USER'), 
                     password=getenv('DB_PASSWORD'), 
                     host=getenv('DB_HOST'), 
                     port=getenv('DB_PORT'))

cursor = connection.cursor()


cursor.execute("""SELECT * FROM trainers;""")

In [ ]:
session.rollback()
cursor.execute("""SELECT * FROM trainer;""")

In [ ]:
session.close()

In [ ]:
with monster_sessions as session:
    monster = session.query(PocketMonster).first()

print(monster)

In [ ]:
dangerous_sessions = SessionManager(
    host=getenv('DB_HOST'),
    user=getenv('DB_USER'),
    password=getenv('DB_PASSWORD'),
    port=getenv('DB_PORT'),
    database=getenv('DB_NAME'),
    expire_on_commit=False
)

with dangerous_sessions as session:
    monster2 = session.query(PocketMonster)\
                     .filter(PocketMonster.name == 'Chooseachu')\
                     .one()
    
print(monster2)

In [ ]:
class Strength(MonsterBase):
    id = Column(Integer, primary_key=True)
    type_id = Column(Integer, ForeignKey('MonsterType.id'), nullable=False)
    strong_against_id = Column(Integer, ForeignKey('MonsterType.id'), nullable=False)

# What about weirder situations?
## Concurrency! Ascynchronous messes!

## Concurrency
* Been well discussed for a long time
* Databases generally are designed for concurrency
* The languages and libraries you're using may not be designed for concurrency...

## Async funtimes
* (often concurrency)
* EVERYTHING is async now (whether it should be or not)
* Some database libraries can handle it, many can't, few are designed specifically for it

### Python async database funtimes
* psycopg2 can actually handle it! Most of the others can't
* (Python 2.7) twisted + sqlalchemy + psycopg2 works fine as long as you keep your sessions untangled
* aiopg: pycopg2 / asyncio library
* asyncpg: fancy new thing with cool blog post, for use with asyncio, high speed, high scale

# aiopg

# asyncpg
A speedy example

In [ ]:
import asyncio
import asyncpg


async def find_monsters(pool):
    """
    Finds monsters! Any monsters, anywhere!
    
    :param pool: Database connection pool
    :type pool: asyncpg.pool.Pool
    """
    print('Looking for monsters...')
    await asyncio.sleep(3)

    async with pool.acquire() as connection:
        async with connection.transaction():
            print('Database transaction started')
            monsters = await connection.fetch('''
                SELECT pocket_monster.name, monster_type.name
                FROM pocket_monster, monster_type
                WHERE pocket_monster.type_id = monster_type.id''')
            for monster_name, monster_type_name in monsters:
                print('I found {} ({} type)!'\
                      .format(monster_name, monster_type_name))

In [ ]:
async def is_it_ready_yet(times=5):
    for _ in range(times):
        await asyncio.sleep(1)
        print('Is it ready yet?')


async def monster_walk():
    print('Waiting for connection pool...')
    pool = await asyncpg.create_pool(
        host=getenv('DB_HOST'),
        user=getenv('DB_USER'),
        password=getenv('DB_PASSWORD'),
        port=getenv('DB_PORT'),
        database=getenv('DB_NAME')
    )
    print('Connection pool ready')
    
    await asyncio.wait([find_monsters(pool), is_it_ready_yet()])

In [ ]:
loop = asyncio.get_event_loop()
loop.run_until_complete(monster_walk())

# What happens when things change?
## Database migrations
In python:
* Alembic for SQLAlchemy
* Django migrations (how many?)
* Lots more

Use it from the beginning, because the only thing that's certain is that everything changes

# connection.close()

The short version: Don't Panic! You'll be fine as long as you always know where your sessions are.

Thank you :)

@betsybookworm